In [1]:
import os

In [22]:
def check_create_dir(path):
    #exists = os.path.isfile(path)
    #exists = os.path.isdir(path)
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)

In [40]:
def remove_low_covered(output_dir, sample_list):
    output_dir = os.path.abspath(output_dir)
    group = output_dir.split("/")[-1]
    uncovered_dir = os.path.join(output_dir, "Uncovered")
    check_create_dir(uncovered_dir)
    sample_list_file = os.path.join(output_dir, "sample_list.txt")
    for root, _, files in os.walk(output_dir):
        #Remove recall gvcf to avoid using them to recalibrate
        if root.endswith('GVCF_recal'):
            for name in files:
                filename = os.path.join(root, name)
                for sample_low in sample_list:
                    if name.startswith(sample_low):
                        os.remove(filename)
        #Place low covered samples in a specific folder to analize them with different parameters
        if root.endswith(group):
            for name in files:
                filename = os.path.join(root, name)
                for sample_low in sample_list:
                    if name.startswith(sample_low) and name.endswith("fastq.gz"):
                        dest_uncovered_path = os.path.join(uncovered_dir, name)
                        os.rename(filename, dest_uncovered_path)
    if os.path.isfile(sample_list_file):
        edit_sample_list(sample_list_file, sample_list)


In [53]:
#os.rename("path/to/current/file.foo", "path/to/new/destination/for/file.foo")

In [42]:
remove_low_covered("/home/laura/ANALYSIS/SNPTB/190920_HRYC", ["HRYC-11986372","10082989-0-COL11"])

/home/laura/ANALYSIS/SNPTB/190920_HRYC/10082989-0-COL11_S18_L001_R2_001.fastq.gz /home/laura/ANALYSIS/SNPTB/190920_HRYC/Uncovered/10082989-0-COL11_S18_L001_R2_001.fastq.gz
/home/laura/ANALYSIS/SNPTB/190920_HRYC/HRYC-11986372_S7_L001_R2_001.fastq.gz /home/laura/ANALYSIS/SNPTB/190920_HRYC/Uncovered/HRYC-11986372_S7_L001_R2_001.fastq.gz
/home/laura/ANALYSIS/SNPTB/190920_HRYC/10082989-0-COL11_S18_L001_R1_001.fastq.gz /home/laura/ANALYSIS/SNPTB/190920_HRYC/Uncovered/10082989-0-COL11_S18_L001_R1_001.fastq.gz
/home/laura/ANALYSIS/SNPTB/190920_HRYC/HRYC-11986372_S7_L001_R1_001.fastq.gz /home/laura/ANALYSIS/SNPTB/190920_HRYC/Uncovered/HRYC-11986372_S7_L001_R1_001.fastq.gz
/home/laura/ANALYSIS/SNPTB/190920_HRYC/GVCF_recal/HRYC-11986372.g.vcf.idx
/home/laura/ANALYSIS/SNPTB/190920_HRYC/GVCF_recal/HRYC-11986372.g.vcf


In [43]:
sample_list = "/home/laura/ANALYSIS/SNPTB/190920_HRYC/sample_list_test.txt"

In [47]:
def edit_sample_list(file_list, sample_list):
    with open(file_list, 'r') as f:
        content = f.read()
        content_list = content.split('\n')
        while '' in content_list : content_list.remove('')
        
    with open (file_list, 'w+') as fout:
            for line in content_list:
                if line not in sample_list:
                    fout.write(line + "\n")
    

In [51]:
edit_sample_list(sample_list, ["HRYC-11986372","10082989-0-COL11", "10082989-0-COL14"])

In [ ]:
def check_reanalysis(output_dir, sample_list):
    output_dir = os.path.abspath(output_dir)
    
    
    #YOUR CODE HERE :D
    #check number of bqrs to avoid recalculation of cohort gvcf 
    #(add if before dvcf cohort to chech if raw vcf exixt and avoid higr recalculation)
    
    
    
    
    
    
    
    
    group = output_dir.split("/")[-1]
    uncovered_dir = os.path.join(output_dir, "Uncovered")
    check_create_dir(uncovered_dir)
    sample_list_file = os.path.join(output_dir, "sample_list.txt")
    for root, _, files in os.walk(output_dir):
        #Remove recall gvcf to avoid using them to recalibrate
        if root.endswith('GVCF_recal'):
            for name in files:
                filename = os.path.join(root, name)
                for sample_low in sample_list:
                    if name.startswith(sample_low):
                        os.remove(filename)